## RDM example
This example employs the model rdm_hgf. The first section simulates beliefs and responses of a putative agent.
The second section shows the use of PAM

In [ ]:
# importing dependencies 

import numpy as np 
import HGF
import pandas as pd
import os,sys

# importing HGF library
import HGF.hgf_config
import HGF.hgf_fit

# Extracting the parent dir in order to append it to the python path 
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))  # Adjust if needed

# Add it to the Python path
sys.path.append(parent_dir)

# importing dependencies 
from PAM.RDM.rdm_hgf_config import rdm_hgf_config
from PAM.RDM.utl.rdm_pdf import rdm_pdf


In [ ]:
# load the trial list -- Specify the path -- 
u = pd.read_csv('u.csv')
u = u.values
u = u.flatten()


(400,)


## Section 1: Simulate dataset

In [ ]:
# Set parameters

om2 = -4 # learning rate
a_a = 2 # intercept of decision threshold "a"
b_a = 1.2 #  Muhat slope effect for "a"
a_v = 2.5 # Intercept of drift rate "v"
b_val = 2.5 # Effect of validity (resp = input) on the drift
b_v = 0 # Muhat slope effect for "v"
Ter = 0 # Non decision time

# Simulate Beliefs
bo = HGF.hgf_fit.fitModel([], u,HGF.hgf_config.ehgf_binary_config, HGF.hgf_config.bayes_optimal_binary_config)
priormus = bo['p_prc']['p'] 
priormus[-2]=om2
esim = HGF.hgf_sim.simModel(u,HGF.hgf.ehgf_binary,priormus)

# Extract trial-wise beliefs about u
muhat = esim['traj']['mu_hat'][:,0]


Ignored trials: []
Irregular trials: []

Initializing optimization run...

Optimization terminated successfully.
         Current function value: 265.698074
         Iterations: 6
         Function evaluations: 21
         Gradient evaluations: 7


RESULTS:

Parameter estimates - perceptual model:
 mu_0: 	 [nan  0.  1.]
 sa_0: 	 [nan 0.1 1. ]
 rho: 	 [nan  0.  0.]
 ka: 	 [1. 1.]
 om: 	 [        nan -3.57818887]
 th: 	 0.6495383397499885

MODEL QUALITY:
 LME: 	 -265.41135491471664 		 (more is better)
 AIC: 	 528.1669974755891 		 (less is better)
 BIC: 	 534.7636322086852 		 (less is better)
Ignored trials: []


C:\Users\fraca\HGF\HGF\hgf.py:39: RuntimeWarning: invalid value encountered in multiply
  mu_hat = np.empty((n, l)) * np.nan     # mu^ quantity
C:\Users\fraca\HGF\HGF\hgf.py:38: RuntimeWarning: invalid value encountered in multiply
  pi = np.empty((n, l)) * np.nan         # pi representation
C:\Users\fraca\HGF\HGF\hgf.py:40: RuntimeWarning: invalid value encountered in multiply
  pi_hat = np.empty((n, l)) * np.nan     # pi^ quantity
C:\Users\fraca\HGF\HGF\hgf.py:37: RuntimeWarning: invalid value encountered in multiply
  mu = np.empty((n, l)) * np.nan         # mu represnetation
C:\Users\fraca\HGF\HGF\hgf.py:41: RuntimeWarning: invalid value encountered in multiply
  v = np.empty((n, l)) * np.nan
C:\Users\fraca\HGF\HGF\hgf.py:43: RuntimeWarning: invalid value encountered in multiply
  da = np.empty((n, l)) * np.nan         # prediction errors


## Section 2: Simulate Responses

In [ ]:
# Calculate trial-wise threshold for both the accumulators
a_c1 = a_a + b_a*(.5-muhat)
a_c0 = a_a + b_a*(.5-(1-muhat))

# Calculate drift rate for both accumulators
drift_c1 = a_v + b_val*(u==1).astype(float) +  b_v * (muhat - .5)
drift_c0 = a_v + b_val*(u==0).astype(float) +  b_v * ((1-muhat) - .5)

# Initialize arrays for response time and response
rt = np.full((len(u), 1), np.nan)
resp = np.full((len(u), 1), np.nan)

# Loop over the trial list    
x = np.arange(0.01, 3.01, 0.01) 

for n in range(len(u)):
    probs_1 =  rdm_pdf(x,drift_c1[n],a_c1[n])
    # Sample from the first accumulator probability distribution
    P1 = np.random.choice(x, p=probs_1/probs_1.sum())
    probs_2 =    rdm_pdf(x,drift_c0[n],a_c0[n])
    P2 = np.random.choice(x, p=probs_2/probs_2.sum())
    P = [P1, P2]
    rt[n, 0], resp[n, 0] = min(P), np.argmin(P) + 1
rt = rt+Ter
resp[resp == 2] = 0

## Section 3: Fit RDM HGF


In [ ]:
# Configure Model
overwrite_option_mus = {'c_prc' : {'om' : bo['p_prc']['om'] },'c_obs': {'b_vsa': 0}}
obs_model =  rdm_hgf_config

# Combine rt and resp for the current subject
y = np.column_stack((rt, resp))  

# Fit the model 
m = HGF.hgf_fit.fitModel(y, u,HGF.hgf_config.ehgf_binary_config , obs_model, opt_model=HGF.hgf_config.quasinewton_optim_config,overwrite_opt=overwrite_option_mus)


Ignored trials: []
Irregular trials: []

Initializing optimization run...



c:\Users\fraca\HGF\PAM\RDM\utl\utl_inverse_gaussian.py:23: RuntimeWarning: overflow encountered in exp
  cdf_part2 = np.exp(2 * drift_cdf * threshold2) * norm.cdf((-drift_cdf * x - threshold2) / np.sqrt(x))
c:\Users\fraca\HGF\PAM\RDM\utl\utl_inverse_gaussian.py:23: RuntimeWarning: invalid value encountered in scalar multiply
  cdf_part2 = np.exp(2 * drift_cdf * threshold2) * norm.cdf((-drift_cdf * x - threshold2) / np.sqrt(x))


Optimization terminated successfully.
         Current function value: -189.266802
         Iterations: 15
         Function evaluations: 168
         Gradient evaluations: 24


RESULTS:

Parameter estimates - perceptual model:
 mu_0: 	 [nan  0.  1.]
 sa_0: 	 [nan 0.1 1. ]
 rho: 	 [nan  0.  0.]
 ka: 	 [1. 1.]
 om: 	 [        nan -3.80141842]
 th: 	 2.2185595341745183

Parameter estimates - observation model:
 a_a: 	 2.03049623486895
 b_a: 	 1.4662918062847088
 a_v: 	 2.526074999508975
 b_val: 	 2.398515988072638
 b_v: 	 0.0
 Ter: 	 0.0

MODEL QUALITY:
 LME: 	 178.94445634143602 		 (more is better)
 AIC: 	 -386.67432903852443 		 (less is better)
 BIC: 	 -386.9087430375718 		 (less is better)


c:\Users\fraca\HGF\.venv\Lib\site-packages\statsmodels\tsa\stattools.py:702: RuntimeWarning: invalid value encountered in divide
  acf = avf[: nlags + 1] / avf[0]
